##### DTU - 02806 Social data analyse og visualisering F17 

## Project Assignment A

** Note: ** We followed the approach to better display too much than too few information. That way everyone is able to follow each step precisely. If some outputs are too long download the notebook from nbviewer by using the icon at the top right. Then open the notebook with Jupyter and select Cell > All Output > Toggle Scrolling.

### 0. Video Introduction

In [ ]:
# IPython global cell magic
%reset
%matplotlib inline

In [45]:
import numpy as np
import pandas as pd
import geoplotlib as gp
from random import randint
from scipy import stats
from collections import Counter
from __future__ import division
from collections import defaultdict
from collections import OrderedDict
import matplotlib.pyplot as plt
import geoplotlib as gp
from geoplotlib.utils import BoundingBox
import plotly.plotly as py
import plotly.tools as pt
import plotly.offline as opy
from plotly.graph_objs import *
from plotly import tools

In [2]:
df = pd.read_csv("NYPD_Motor_Vehicle_Collisions_reduced_data.csv", low_memory=False)

In [3]:
# Drop NaN values from Lat and Lon
df = df.dropna(subset=['LATITUDE','LONGITUDE'])

In [5]:
df.head(5)

,Date,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,Maximum Humidity,Mean Temperature,Min Temperature,Minimum Humidity,Precipitation,Sea Level Pressure,Snow,Snow Depth,Visibility,Wind Speed
0,2017/03/28,0:00,BRONX,10466,40.894600,-73.861206,"(40.8946, -73.861206)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
1,2017/03/28,0:00,BROOKLYN,11218,40.654080,-73.977610,"(40.65408, -73.97761)",18 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
2,2017/03/28,0:00,BROOKLYN,11235,40.583847,-73.940590,"(40.583847, -73.94059)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
3,2017/03/28,0:00,MANHATTAN,10019,40.762770,-73.975590,"(40.76277, -73.97559)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
4,2017/03/28,0:00,QUEENS,11429,40.713715,-73.731440,"(40.713715, -73.73144)",222 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9


In [6]:
print "Columns of Dataset: ", df.columns

Columns of Dataset:  Index([u'Date', u'TIME', u'BOROUGH', u'ZIP CODE', u'LATITUDE', u'LONGITUDE',
       u'LOCATION', u'ON STREET NAME', u'NUMBER OF PERSONS INJURED',
       u'NUMBER OF PERSONS KILLED', u'NUMBER OF PEDESTRIANS INJURED',
       u'NUMBER OF PEDESTRIANS KILLED', u'NUMBER OF CYCLIST INJURED',
       u'NUMBER OF CYCLIST KILLED', u'NUMBER OF MOTORIST INJURED',
       u'NUMBER OF MOTORIST KILLED', u'CONTRIBUTING FACTOR VEHICLE 1',
       u'CONTRIBUTING FACTOR VEHICLE 2', u'UNIQUE KEY', u'VEHICLE TYPE CODE 1',
       u'VEHICLE TYPE CODE 2', u'Average Humidity', u'Dew Point', u'Events',
       u'Max Gust Speed', u'Max Temperature', u'Max Wind Speed',
       u'Maximum Humidity', u'Mean Temperature', u'Min Temperature',
       u'Minimum Humidity', u'Precipitation', u'Sea Level Pressure', u'Snow',
       u'Snow Depth', u'Visibility', u'Wind Speed'],
      dtype='object')


In [7]:
df = df[(np.abs(stats.zscore(df[['LATITUDE','LONGITUDE']])) < 1).all(axis=1)]
print "The size of Dataset: ", df.shape

The size of Dataset:  (802922, 37)


### 1. Preliminary Analysis

In [8]:
years=sorted(Counter(df.Date.str[0:4]).keys())
dirty_dis=sorted(Counter(df.BOROUGH).keys())
districts = [x for x in dirty_dis if str(x) != 'nan']

index = np.arange(len(years))
bar_width = 0.1
opacity = 0.4


In [9]:
year_total_dict=defaultdict(list)
data=list()
colors = list()
for i in range(len(years)):
    colors.append('%06X' % randint(0, 0xFFFFFF))

for district in districts:
    for year in years:
        year_df=df.loc[(df['Date'].str[0:4] == year) & (df['BOROUGH']==district)]
        year_total_dict[district].append(year_df.size)
    district_bar = Bar(x=years,y = year_total_dict[district],name=district,marker=dict(color=colors[color_index]),opacity=opacity)
    data.append(district_bar)



layout = Layout(title="Total of Accidents in each district in last 5 years",
                xaxis=dict(title='Years'),
                yaxis=dict(title='Accident Total'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [44]:
year_kill_dict=defaultdict(list)
year_injur_dict=defaultdict(list)
data=list()
for district in districts:
    for year in years:
        year_injur_df=df.loc[(df['Date'].str[0:4] == year) & (df['BOROUGH']==district) & (df['NUMBER OF PERSONS INJURED']!=0)]
        year_kill_df=df.loc[(df['Date'].str[0:4] == year) & (df['BOROUGH']==district) & (df['NUMBER OF PERSONS KILLED']!=0)]
        year_injur_dict[district].append(year_injur_df.size)
        year_kill_dict[district].append(year_kill_df.size)
    district_bar_injur = Bar(x=years,y = year_injur_dict[district],name=district,marker=dict(color='rgb(250,130,0)'),opacity=opacity,width = 0.15)
    #district_bar_kill = Bar(x=years,y = year_kill_dict[district],name=district,marker=dict(color='rgb(250,130,0)'),opacity=opacity,width = 0.15,offset=-0.15)
    data.append(district_bar_injur)
    #data.append(district_bar_kill)
    #data.append(district_bar_kill)

layout = Layout(title="Total of Injury in each district in last 5 years",
                xaxis=dict(title='Years'),
                yaxis=dict(title='Accident Total'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [47]:
year_kill_dict=defaultdict(list)
data=list()
for district in districts:
    for year in years:
        year_kill_df=df.loc[(df['Date'].str[0:4] == year) & (df['BOROUGH']==district) & (df['NUMBER OF PERSONS KILLED']!=0)]
        year_kill_dict[district].append(year_kill_df.size)
    #district_bar_injur = Bar(x=years,y = year_injur_dict[district],name=district,marker=dict(color='rgb(250,130,0)'),opacity=opacity,width = 0.15)
    district_bar_kill = Bar(x=years,y = year_kill_dict[district],name=district,marker=dict(color='rgb(250,20,0)'),opacity=opacity,width = 0.15)
    data.append(district_bar_kill)
    #data.append(district_bar_kill)
    #data.append(district_bar_kill)

layout = Layout(title="Total of Killed in each district in last 5 years",
                xaxis=dict(title='Years'),
                yaxis=dict(title='Accident Total'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [88]:
year16_df.sort('Date').drop_duplicates('Date').Precipitation.values

C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



array(['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 'T',
       '45.72', '0.00', 'T', '0.00', 'T', 'T', '6.10', '1.27', 'T', '0.00',
       '0.00', '0.00', '0.25', '58.67', 'T', '0.00', '0.00', 'T', '0.00',
       '0.00', '0.00', '0.00', '0.25', '0.00', '18.54', 'T', '13.46',
       '0.00', '0.00', '1.27', 'T', '0.25', 'T', '0.00', '0.00', '0.00',
       '11.18', '25.65', '0.00', '0.00', '0.00', '0.00', '0.76', '0.00',
       '7.62', '30.99', '0.51', '0.00', '0.00', '0.00', '1.27', '0.00',
       '3.56', '0.00', '2.79', '0.00', '0.00', '0.00', '0.00', '0.00',
       '0.00', '1.52', '0.00', 'T', '7.37', '0.00', '0.51', 'T', '0.00',
       '0.00', '1.78', '1.52', '0.00', '0.00', '0.00', '1.02', '0.00',
       '0.00', '9.65', '0.00', '0.00', '0.00', '0.25', '3.81', '2.29',
       '11.94', '0.00', '0.00', '2.29', '0.25', '2.29', '0.00', '0.25',
       '5.08', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00',
       '0.00', '0.00', 'T', '4.06', '0.00', '0.00', '6.10

In [92]:
#accident in year 2016 only
year16_df=df.loc[(df['Date'].str[0:4] == '2014')]
days=year16_df.sort('Date').drop_duplicates('Date').Date.values
days16_acci=year16_df.sort('Date').groupby('Date').size().values
days16_visi=year16_df.sort('Date').drop_duplicates('Date').Visibility.fillna(0).values*10
# Create traces
trace0 = Scatter(
    x = days,
    y = days16_acci,
    mode = 'lines',
    name = 'Accidents per Day'
)

trace1 = Scatter(
    x = days,
    y = days16_visi,
    fill='tozeroy',
    mode= 'none',
    name='Visibility per Day'
)
data=[trace0,trace1]
layout = Layout(title="Daily Car Accidents and Visibility in 2014",
                xaxis=dict(title='Date'),
                yaxis=dict(title='Accident Total/Visibility Trend'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)

C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)

C:\WorkingGround\Anaconda\lib\site-packages\ipykernel\__main__.py:5: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



### Plotting all crashes

In [ ]:
# Selecting location for each type of crime
lat_list_all = df['LATITUDE']
lon_list_all = df['LONGITUDE']
    
geo_data_for_plotting = {"lat": lat_list_all,
                         "lon": lon_list_all}

max_lat, min_lat = max(lat_list_all), min(lat_list_all)
max_lon, min_lon = max(lon_list_all), min(lon_list_all)

bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
gp.set_bbox(bbox)
    
# And plotting it
gp.dot(geo_data_for_plotting, color='green', point_size=0.25)


gp.inline()

### Plotting injuries/fatal accidents

In [ ]:
# Selecting location for each type of crime
lat_list_casu = df[(df['NUMBER OF PERSONS KILLED'] > 0) |(df['NUMBER OF PERSONS INJURED'] > 0)]['LATITUDE']
lon_list_casu = df[(df['NUMBER OF PERSONS KILLED'] > 0) |(df['NUMBER OF PERSONS INJURED'] > 0)]['LONGITUDE']
    
geo_data_for_plotting = {"lat": lat_list_casu,
                         "lon": lon_list_casu}

max_lat, min_lat = max(lat_list_casu), min(lat_list_casu)
max_lon, min_lon = max(lon_list_casu), min(lon_list_casu)

bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
gp.set_bbox(bbox)
    
# And plotting it
gp.dot(geo_data_for_plotting, color='green', point_size=1)


gp.inline()

* KDE plot

In [ ]:
# And plotting it

bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
gp.set_bbox(bbox)

gp.kde(geo_data_for_plotting, bw=4, alpha=160, cut_below=1e-2)


gp.inline()

### Cluster analysis of casualties

In [ ]:
from sklearn import cluster

kmeans_data = np.array([lat_list_casu, lon_list_casu]).T
kmeans_cluster_n = range(2,11)
kmeans_inertias = []

for k in kmeans_cluster_n:
    
    kmeans = cluster.KMeans(n_clusters=k)
    kmeans.fit(kmeans_data)
    current_inertia = kmeans.inertia_
    kmeans_inertias.append(current_inertia)
    
    print("Inertia of cluster {} is: \t{}".format(k, current_inertia))

In [ ]:
fig = plt.figure(figsize=(6,4))

ax = fig.add_subplot(111)

ax.plot(list(kmeans_cluster_n), kmeans_inertias, marker='o', linestyle='-', c='black')

ax.set_title('Inertia as function of cluster number in K-means')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

plt.show()

Judging by the "elbow rule", the optimal clustering seems to be around 5 clusters.

As the optimal number of clusters has been found by the elbow rule - the clustering can now be visualized by applying the clustering to the locations of car accidents involving casualties in NYC.

First the locations of the accidents are visualized without a map.

In [ ]:
fig = plt.figure(figsize = (8,8))

ax = fig.add_subplot(111)

ax.scatter(y=lat_list_casu, x=lon_list_casu, s=0.1)
ax.set_xlim(min_lon-0.01, max_lon+0.01)
ax.set_ylim(min_lat-0.01, max_lat+0.01)

ax.set_title("Locations of car accidents involving casualties in NYC", fontsize=14)
ax.set_xlabel("Longitude", fontsize = 12)
ax.set_ylabel("Latitude", fontsize = 12)

plt.show()

In [ ]:
kmeans_data = np.array([lon_list_casu, lat_list_casu]).T

k = 5
kmeans = cluster.KMeans(n_clusters=k)
kmeans.fit(kmeans_data)

In [ ]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
labels.shape

In [ ]:
fig = plt.figure(figsize = (8,8))

ax = fig.add_subplot(111)

for i in range(k):
    
    # select only data observations with cluster label == i
    ds = kmeans_data[np.where(labels==i)]
    # plot the data observations
    ax.plot(ds[:,0],ds[:,1],'o', markersize=0.1)
    # plot the centroids
    lines = ax.plot(centroids[i,0],centroids[i,1],'kx', markersize=12, mew=2.5)

ax.set_title("Locations of car accidents involving casualties in NYC", fontsize=14)
ax.set_xlabel("Longitude", fontsize = 12)
ax.set_ylabel("Latitude", fontsize = 12)   
    
plt.show()

### Stability analysis of clustering

First, we initialize the grid of the actual distances and their location on the grid.

In [ ]:
#for even spacing of gridpoints
lat_step = (max_lat - min_lat)/20 
lon_step = (max_lon - min_lon)/20  

grid_center_locations = np.zeros([20,20,2]) # to hold the lat-lon locations of points on the grid
grid_points_locations = np.zeros([20,20,2]) # to hold the grid-wise indexes (e.g. [12,15])

for idx_row, i in enumerate(np.arange(min_lon, max_lon, lon_step)): # loop thru rows (latitutes)
    for idx_col, j in enumerate(np.arange(min_lat, max_lat, lat_step)): # loop thru columns (longitudes)
        
        grid_center_locations[idx_row,idx_col] = [i, j] # assign the actual location
        grid_points_locations[idx_row,idx_col] = idx_row, idx_col # assing the grid point index
        
grid_center_locations = grid_center_locations.reshape(400,2)
grid_points_locations = grid_points_locations.reshape(400,2)

Next, a function is defined to calculate which point on the grid each centroid belongs to

In [ ]:
def cluster_grid_locator(k):
    
    N_center_locs = {} # to hold the grid locations for each cluster

    # each key is the cluster number k, with values as it's location for subsample N (10 in our case)
    for cluster_idx in range(1,k+1):
        N_center_locs[cluster_idx] = []
        
    # we do 10 iterations of random samplings 
    for N in range(10):
        
        N_rand_idx = np.random.randint(kmeans_data.shape[0], size=no_samples) # grab random samples

        kmeans = cluster.KMeans(n_clusters=k) # initialize model
        kmeans.fit(kmeans_data[N_rand_idx]) # fit model on random sample

        current_centers = kmeans.cluster_centers_ # grab the k centroid locations

        # for each center, we find it's grid point index
        for cluster_idx, center in enumerate(current_centers): 
            
            # grab the distance of current cent to all grid points
            center_diff = np.abs(grid_center_locations - center)
            # calculate Chebyshev distance between centroids and all grid points
            center_distances = np.apply_along_axis(np.max, 1, center_diff)
            # grab index of the closest grid point
            center_min_idx = np.argmin(center_distances)
            # grab the coords of closest grid point
            center_min_loc = grid_points_locations[center_min_idx] 
            
            # add the coordinates to the dictionary of center locations
            N_center_locs[cluster_idx+1].append(center_min_loc)
    
    return N_center_locs

Lastly, the intra cosine similarities of of each centroid locations (for all Ns) is calculated to see how well centroid locations converge for different subsamples of the data

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

kmeans_data = np.array([lon_list_casu, lat_list_casu]).T
no_samples = int(kmeans_data.shape[0]/2) # to grab 50% of the data in each clustering

clustering_ave_sim = {}

# loop through all cluster numbers
for cluster_no in range(2,11):
    
    # grab the locations of all centroids for N=10
    cluster_no_locs = cluster_grid_locator(cluster_no)
    
    # mean similarities of each centroid location (for all Ns)
    ave_cosine_similarities = []
    
    # loop through all clusters in clustering k
    for sim_collection in cluster_no_locs:
        
        # grab converged locations for all Ns
        current_sim = cluster_no_locs[sim_collection]
        
        # grab their average cosine similarity
        current_ave_cos_dist = np.mean(cosine_similarity(current_sim))
        
        # add it to a list of similarities for each k
        ave_cosine_similarities.append(current_ave_cos_dist)
        

    # the total similarity of the current clustering
    clustering_ave_similarity = np.mean(ave_cosine_similarities)
    
    clustering_ave_sim[cluster_no] = clustering_ave_similarity

As can be seen below, in this instance the amount of clusters that shows the highest similarity for different subsets of the data is K=2. It should be noted that calculating the similarity for different subsets (by running the code above) switches between the highest similarity being K=2 and K=3. Therefore, both those clusterings can be considered good for describing the dataset.

In [ ]:
for i in clustering_ave_sim:
    
    print("{}: {:.4f}".format(i, clustering_ave_sim[i]))

### Plotting clustered data on a map

In [ ]:
k = 2
kmeans = cluster.KMeans(n_clusters=k)
kmeans.fit(kmeans_data)

In [ ]:
k_means_idxs = kmeans.labels_ + 1
k_means_centroids = kmeans.cluster_centers_

In [ ]:
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
gp.set_bbox(bbox)


colors = ['blue', 'green', 'red', 'purple', 'orange']

for cluster in range(1,3):
    
    current_idx = np.where(k_means_idxs == cluster)
    
    current_lat = np.array(lat_list_casu)[current_idx]
    current_lon = np.array(lon_list_casu)[current_idx]

    geo_data_for_plotting = {"lat": current_lat,
                             "lon": current_lon}
    
    geo_center_for_plotting = {"lat": k_means_centroids[cluster-1][1],
                               "lon": k_means_centroids[cluster-1][0]}
    
    # And plotting it, note that shape cannot be changed and neither can the size for centroids only
    gp.dot(geo_data_for_plotting, color=colors[cluster-1], point_size=0.1)
    gp.dot(geo_center_for_plotting, color=(199,21,133,255), point_size=0.1)
    
    
gp.inline()

As can be expected, the clustering is quite "arbitrary" - as preliminary plotting of the occurences did not seem to show any inherent "hot spots" on the map.